# Đọc file final_report.csv

In [3]:
import pandas as pd

df = pd.read_csv("final_report.csv")

df["order_date"] = pd.to_datetime(df["order_date"])

df["month"] = df["order_date"].dt.to_period("M").astype(str)

# 1.Pivot doanh thu theo category × tháng

In [4]:
df["revenue"] = df["quantity"] * df["unit_price"]

pivot_revenue = pd.pivot_table(
    df,
    values="revenue",
    index="category",
    columns="month",
    aggfunc="sum",
    fill_value=0
)

pivot_revenue

month,2024-01,2024-02,2024-03,2024-04
category,,,,
gia dụng,5400000.0,10100000.0,21250000.0,2950000.0
điện tử,18950000.0,20100000.0,11950000.0,8450000.0


# 2.Pivot tỷ lệ trả hàng theo category × sản phẩm.

In [9]:
df["is_return"] = (df["return_merge"] == "both").astype(int)

pivot_return_rate = pd.pivot_table(
    df,
    values="is_return",
    index="category",
    columns="product_name",
    aggfunc="mean",
    fill_value=0
)

pivot_return_rate

product_name,ban phim co,chuot khong day,loa bluetooth,sac du phong,tai nghe bluetooth
category,,,,,
gia dụng,0.00,0.115385,0.000,0.233333,0.000000
điện tử,0.15,0.000000,0.125,0.000000,0.090909


# 3.Thực hiện stack/unstack bảng pivot.

In [10]:
revenue_stack = pivot_revenue.stack()
revenue_stack
revenue_unstack = revenue_stack.unstack()
revenue_unstack
return_stack = pivot_return_rate.stack()
return_stack

category  product_name      
gia dụng  ban phim co           0.000000
          chuot khong day       0.115385
          loa bluetooth         0.000000
          sac du phong          0.233333
          tai nghe bluetooth    0.000000
điện tử   ban phim co           0.150000
          chuot khong day       0.000000
          loa bluetooth         0.125000
          sac du phong          0.000000
          tai nghe bluetooth    0.090909
dtype: float64

# 4.Nhận xét danh mục có hiệu suất thấp nhất.

In [ ]:
category_perf = pivot_revenue.sum(axis=1)

low_perf_category = category_perf.idxmin()

print("Danh mục có hiệu suất thấp nhất:", low_perf_category)

Danh mục có hiệu suất thấp nhất: điện tử
